# Financial Service - Balance Sheet, Income, Cash Flow

Access comprehensive financial statements for fundamental analysis and valuation.

## Overview

The **Financial Service** provides three types of financial statements:

- **Balance Sheet**: Assets, liabilities, equity
- **Income Statement**: Revenue, expenses, profit/loss
- **Cash Flow Statement**: Operating, investing, financing activities

### Features

- **Multi-Period Data**: Multiple quarters/years for trend analysis
- **Hierarchical Structure**: Account hierarchy with levels (-1 for totals, 0+ for details)
- **Audit Status**: Audited, Reviewed, or Unaudited flags
- **Thai/English Support**: Financial data in both languages
- **Quarter Codes**: Q1-Q4, 6M (half-year), Q9 (full-year)

### When to Use

- Fundamental analysis and valuation
- Financial ratio calculations
- Trend analysis (revenue growth, margin improvement)
- Credit analysis (debt levels, coverage ratios)
- DCF modeling and forecasting

In [ ]:
!pip install settfex

In [ ]:
import asyncio
from settfex.services.set import get_balance_sheet, get_income_statement, get_cash_flow
import pandas as pd

print("Libraries imported!")

## Basic Usage - Balance Sheet

In [ ]:
# Fetch balance sheet
balance_sheets = await get_balance_sheet("CPALL")

print(f"Balance Sheets for CPALL: {len(balance_sheets)} periods\n")

# Get latest period
latest = balance_sheets[0]
print(f"Latest Period: {latest.quarter} {latest.year}")
print(f"Status: {latest.status}")
print(f"Period: {latest.begin_date} to {latest.end_date}")
print(f"\nKey Line Items (in thousands THB):\n")

# Find key accounts (level -1 are totals)
key_accounts = [acc for acc in latest.accounts if acc.level == -1]
for acc in key_accounts:
    print(f"{acc.account_name:40} {acc.amount:>15,.0f}")

## Basic Usage - Income Statement

In [ ]:
# Fetch income statement
income_statements = await get_income_statement("CPALL")

latest_income = income_statements[0]
print(f"Income Statement: {latest_income.quarter} {latest_income.year}\n")

# Find key metrics
key_items = [acc for acc in latest_income.accounts if acc.level == -1]
for item in key_items:
    print(f"{item.account_name:40} {item.amount:>15,.0f}")

## Basic Usage - Cash Flow

In [ ]:
# Fetch cash flow
cash_flows = await get_cash_flow("CPALL")

latest_cf = cash_flows[0]
print(f"Cash Flow Statement: {latest_cf.quarter} {latest_cf.year}\n")

# Find operating, investing, financing cash flows
key_cf = [acc for acc in latest_cf.accounts if acc.level == -1]
for item in key_cf:
    print(f"{item.account_name:40} {item.amount:>15,.0f}")

## Advanced Usage - Financial Ratio Analysis

In [ ]:
def calculate_financial_ratios(balance_sheet, income_statement):
    """
    Calculate key financial ratios from statements.
    """
    # Helper to find account by name
    def find_account(accounts, keywords):
        for acc in accounts:
            if any(kw.lower() in acc.account_name.lower() for kw in keywords):
                return acc.amount
        return None
    
    # Extract key figures
    total_assets = find_account(balance_sheet.accounts, ["Total Assets"])
    total_liabilities = find_account(balance_sheet.accounts, ["Total Liabilities"])
    total_equity = find_account(balance_sheet.accounts, ["Total Equity", "Shareholders"])
    
    net_income = find_account(income_statement.accounts, ["Net Income", "Net Profit"])
    total_revenue = find_account(income_statement.accounts, ["Total Revenue", "Total Income"])
    
    ratios = {}
    
    # Calculate ratios
    if total_assets and total_liabilities:
        ratios['debt_to_assets'] = (total_liabilities / total_assets) * 100
    
    if total_assets and total_equity:
        ratios['equity_ratio'] = (total_equity / total_assets) * 100
    
    if total_equity and total_liabilities:
        ratios['debt_to_equity'] = (total_liabilities / total_equity) * 100
    
    if total_revenue and net_income:
        ratios['net_profit_margin'] = (net_income / total_revenue) * 100
    
    if total_assets and net_income:
        ratios['roa'] = (net_income / total_assets) * 100
    
    if total_equity and net_income:
        ratios['roe'] = (net_income / total_equity) * 100
    
    return ratios

# Calculate ratios
ratios = calculate_financial_ratios(latest, latest_income)

print(f"Financial Ratios for CPALL ({latest.quarter} {latest.year}):\n")
for ratio_name, value in ratios.items():
    print(f"{ratio_name:25} {value:>8.2f}%")

## Use Case: Trend Analysis

In [ ]:
# Analyze revenue trend
def extract_revenue_trend(income_statements):
    """
    Extract revenue across periods.
    """
    trend = []
    for stmt in income_statements:
        for acc in stmt.accounts:
            if "Total Revenue" in acc.account_name or "Total Income" in acc.account_name:
                trend.append({
                    "period": f"{stmt.quarter} {stmt.year}",
                    "revenue": acc.amount / 1_000_000,  # Convert to millions
                    "year": stmt.year,
                    "quarter": stmt.quarter
                })
                break
    return pd.DataFrame(trend)

revenue_trend = extract_revenue_trend(income_statements)
print("Revenue Trend (in millions THB):\n")
print(revenue_trend.to_string(index=False))

# Calculate growth
if len(revenue_trend) >= 2:
    latest_rev = revenue_trend.iloc[0]['revenue']
    prev_rev = revenue_trend.iloc[1]['revenue']
    growth = ((latest_rev - prev_rev) / prev_rev) * 100
    print(f"\nRevenue Growth (QoQ): {growth:+.2f}%")

## Use Case: Comparative Company Analysis

In [ ]:
async def compare_companies(symbols: list[str]):
    """
    Compare financial metrics across companies.
    """
    # Fetch balance sheets and income statements
    bs_tasks = [get_balance_sheet(symbol) for symbol in symbols]
    is_tasks = [get_income_statement(symbol) for symbol in symbols]
    
    balance_sheets = await asyncio.gather(*bs_tasks, return_exceptions=True)
    income_stmts = await asyncio.gather(*is_tasks, return_exceptions=True)
    
    comparison = []
    for symbol, bs_list, is_list in zip(symbols, balance_sheets, income_stmts):
        if isinstance(bs_list, Exception) or isinstance(is_list, Exception):
            continue
        
        if bs_list and is_list:
            ratios = calculate_financial_ratios(bs_list[0], is_list[0])
            ratios['symbol'] = symbol
            ratios['period'] = f"{bs_list[0].quarter} {bs_list[0].year}"
            comparison.append(ratios)
    
    df = pd.DataFrame(comparison)
    return df

# Compare retail companies
retail_stocks = ["CPALL", "MAKRO", "HMPRO"]
comparison = await compare_companies(retail_stocks)

print("Retail Sector Financial Comparison:\n")
if not comparison.empty:
    print(comparison[['symbol', 'period', 'net_profit_margin', 'roe', 'debt_to_equity']].to_string(index=False))

## Use Case: Export to Excel

In [ ]:
# Export financial statements to CSV
def export_balance_sheet_to_csv(balance_sheets, filename="balance_sheet.csv"):
    """
    Export balance sheet data to CSV.
    """
    data = []
    for bs in balance_sheets:
        for acc in bs.accounts:
            data.append({
                "period": f"{bs.quarter} {bs.year}",
                "year": bs.year,
                "quarter": bs.quarter,
                "account_code": acc.account_code,
                "account_name": acc.account_name,
                "amount": acc.amount,
                "level": acc.level,
                "status": bs.status
            })
    
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False, encoding="utf-8")
    return df

# Export CPALL balance sheets
bs_df = export_balance_sheet_to_csv(balance_sheets, "cpall_balance_sheet.csv")
print(f"Exported {len(bs_df)} line items to cpall_balance_sheet.csv")

## Error Handling

In [ ]:
async def safe_get_financials(symbol: str):
    try:
        bs = await get_balance_sheet(symbol)
        income = await get_income_statement(symbol)
        cf = await get_cash_flow(symbol)
        
        print(f"✓ {symbol}: {len(bs)} balance sheets, {len(income)} income statements, {len(cf)} cash flows")
        return bs, income, cf
    except Exception as e:
        print(f"✗ {symbol}: {e}")
        return None, None, None

await safe_get_financials("PTT")

## Next Steps

Now you have mastered all SET services! Consider:

1. **Build a complete stock analysis system** combining all services
2. **Create automated screening systems** for value/growth stocks
3. **Develop portfolio management tools** with real-time data
4. **Build financial models** using historical financial data
5. **Create dashboards** with matplotlib, plotly, or streamlit

### Related Notebooks

- **[Stock List](01_stock_list.ipynb)** - Universe definition
- **[Highlight Data](02_highlight_data.ipynb)** - Current valuation
- **[Price Performance](10_price_performance.ipynb)** - Relative performance
- **[Trading Statistics](09_trading_statistics.ipynb)** - Historical trends

**Documentation**: `/docs/settfex/services/set/financial.md`